In [3]:
import numpy as np
from scipy.optimize import minimize

# =====================================================
# 1. PARÁMETROS DEL PROBLEMA
# =====================================================
# Coeficientes de la restricción presupuestaria
# Estos provienen de la formulación teórica

a1 = 4820/297
a2 = 320/720
a3 = 210/1050
a4 = 70/815


# =====================================================
# 2. FUNCIÓN OBJETIVO
# =====================================================
# Esta función NO es el ingreso directamente.
# Aquí estamos minimizando el error de las
# condiciones de primer orden (FOC).
#
# Si el valor final es cercano a 0,
# entonces las FOC se satisfacen.

def objective(vars):
    
    # Extraemos variables del vector
    p1, p2, p3, p4, lam = vars
    
    # -------------------------------
    # Demandas isoelásticas
    # -------------------------------
    q1 = 4820*(p1/297)**(-0.4)
    q2 = 320*(p2/720)**(-2.7)
    q3 = 210*(p3/1050)**(-1.1)*(p4/815)**(0.1)
    q4 = 70*(p4/815)**(-0.4)*(p3/1050)**(0.4)
    
    # -------------------------------
    # Condiciones de Primer Orden
    # -------------------------------
    F1 = 0.6*q1 + lam*a1
    F2 = -1.7*q2 + lam*a2
    F3 = -0.1*q3 + 0.4*q4 + lam*a3
    F4 = 0.6*q4 + 0.1*q3 + lam*a4
    
    # -------------------------------
    # Función a minimizar:
    # suma de cuadrados de las FOC
    # -------------------------------
    return F1**2 + F2**2 + F3**2 + F4**2


# =====================================================
# 3. RESTRICCIÓN EXACTA
# =====================================================
# Se impone como restricción de igualdad
# para que se cumpla exactamente.

def constraint(vars):
    p1, p2, p3, p4, lam = vars
    return a1*p1 + a2*p2 + a3*p3 + a4*p4 - 5420


cons = {'type':'eq','fun':constraint}


# =====================================================
# 4. LÍMITES DE LAS VARIABLES
# =====================================================
# Precios positivos y acotados
# Lambda acotado para estabilidad numérica

bounds = [(1e-3,6000)]*4 + [(-1000,1000)]


# =====================================================
# 5. PUNTO INICIAL
# =====================================================

initial_guess = [250,3000,1800,1600,-200]


# =====================================================
# 6. EJECUCIÓN DEL ALGORITMO SLSQP
# =====================================================

solution = minimize(
    objective,
    initial_guess,
    method='SLSQP',
    bounds=bounds,
    constraints=cons,
    options={'disp':True,'ftol':1e-14,'maxiter':2000}
)


# =====================================================
# 7. EXTRAER RESULTADOS
# =====================================================

p1, p2, p3, p4, lam = solution.x

# Recalcular cantidades óptimas
q1 = 4820*(p1/297)**(-0.4)
q2 = 320*(p2/720)**(-2.7)
q3 = 210*(p3/1050)**(-1.1)*(p4/815)**(0.1)
q4 = 70*(p4/815)**(-0.4)*(p3/1050)**(0.4)

# Calcular ganancia total
R = p1*q1 + p2*q2 + p3*q3 + p4*q4


# =====================================================
# 8. IMPRIMIR RESULTADOS
# =====================================================

print("\n========= RESULTADOS ÓPTIMOS =========\n")

print("Precios óptimos:")
print("p1 =",p1)
print("p2 =",p2)
print("p3 =",p3)
print("p4 =",p4)

print("\nCantidades óptimas:")
print("q1 =",q1)
print("q2 =",q2)
print("q3 =",q3)
print("q4 =",q4)

print("\nMultiplicador de Lagrange:")
print("Lambda =",lam)

print("\nGanancia total óptima:")
print("R =",R)

print("\nError total FOC:")
print(solution.fun)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 11693.19758974229
            Iterations: 76
            Function evaluations: 710
            Gradient evaluations: 76

========= RESULTADOS ÓPTIMOS =========

Precios óptimos:
p1 = 201.29346062244778
p2 = 3596.4384547810178
p3 = 2008.019114614797
p4 = 1783.6429203405264

Cantidades óptimas:
q1 = 5631.417828696497
q2 = 4.159983522471612
q3 = 111.30070811697729
q4 = 66.32378994600454

Multiplicador de Lagrange:
Lambda = -208.02182520530317

Ganancia total óptima:
R = 1490320.6154171368

Error total FOC:
11693.19758974229
